# Initialization

In [ ]:
# Standard includes
%matplotlib inline
import pickle

import boost_histogram as bh
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Style setup
import seaborn as sns

sns.set_palette("muted")
sns.set_color_codes()
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})
sns.set_style({"axes.grid": "True", "grid.color": "0.95"})

plt.rcParams["figure.figsize"] = [6, 6]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["axes.formatter.min_exponent"] = 0

In [ ]:
import mplhep as hep

hep.set_style("LHCb2")

plt.rcParams["font.size"] = 28
plt.rcParams["figure.dpi"] = 50  # Comment out/set to 300 for production plots
plt.rcParams["axes.formatter.min_exponent"] = 0

# Data

Efficiency histograms for the "Efficiency vs Momentum" plots can be made by running:
```
pidcalib2.make_eff_hists --sample Turbo18 --magnet up --particle K --pid-cut "DLLK>0" --pid-cut "DLLK>5" --output-dir pidcalib_output_many --bin-var P
pidcalib2.make_eff_hists --sample Turbo18 --magnet up --particle Pi --pid-cut "DLLK>0" --pid-cut "DLLK>5" --output-dir pidcalib_output_many --bin-var P
```

In [ ]:
hists = {}
particles = ["K", "Pi"]
cuts = ["DLLK>0", "DLLK>5"]

for particle in particles:
    for cut in cuts:
        with open(
            f"../pidcalib_output_many/effhists-Turbo18-up-{particle}-{cut}-P.pkl", "rb"
        ) as f:
            hists[f"{particle}_{cut}"] = pickle.load(f)

Efficiency histograms for the "ID Efficiency vs Mis-ID Efficiency" plots can be made by running:
```
pidcalib2.make_eff_hists --sample Turbo18 --magnet up --particle K $(for I in $(seq -20 20); do echo "--pid-cut DLLK>${I} "; done) --output-dir pidcalib_output_many --bin-var P
pidcalib2.make_eff_hists --sample Turbo18 --magnet down --particle K $(for I in $(seq -20 20); do echo "--pid-cut DLLK>${I} "; done) --output-dir pidcalib_output_many --bin-var P
pidcalib2.make_eff_hists --sample Turbo18 --magnet up --particle Pi $(for I in $(seq -20 20); do echo "--pid-cut DLLK>${I} "; done) --output-dir pidcalib_output_many --bin-var P
pidcalib2.make_eff_hists --sample Turbo18 --magnet down --particle Pi $(for I in $(seq -20 20); do echo "--pid-cut DLLK>${I} "; done) --output-dir pidcalib_output_many --bin-var P
```

In [ ]:
hists2 = {}
particles = ["K", "Pi"]
cuts2 = [f"DLLK>{cut}" for cut in range(-20, 21)]
mags = ["up", "down"]

for mag in mags:
    for particle in particles:
        for cut in cuts2:
            with open(
                f"../pidcalib_output_many2/effhists-Turbo18-{mag}-{particle}-{cut}-P.pkl",
                "rb",
            ) as f:
                hists2[f"eff_{particle}_{mag}_{cut}"] = pickle.load(f)
                hists2[f"passing_{particle}_{mag}_{cut}"] = pickle.load(f)
                hists2[f"total_{particle}_{mag}_{cut}"] = pickle.load(f)

# Plots

In [ ]:
plots_save = True
plots_format = ".pdf"

## Efficiency vs Momentum

In [ ]:
colors = {
    "K_DLLK>0": "xkcd:light salmon",
    "K_DLLK>5": "xkcd:red",
    "Pi_DLLK>0": "xkcd:pastel blue",
    "Pi_DLLK>5": "xkcd:blue",
}
for name, hist in hists.items():
    plt.hist(
        hist.axes[0].edges[:-1],
        bins=hist.axes[0].edges,
        weights=hist.values(),
        histtype="stepfilled",
        label=name.replace("_", " ").replace("pi", r"$\pi$"),
        color=colors[name],
        edgecolor=colors[name],
        linewidth=1.5,
        fc=(*mpl.colors.to_rgb(colors[name]), 0.03),
    )
plt.ylim(top=1.35)
plt.margins(x=-0.01)
plt.legend()
plt.xlabel("Momentum [MeV/c]")
plt.ylabel("Efficiency")
plt.figtext(0.2, 0.8, "LHCb\n $\\sqrt{s}$=13 TeV 2018 Validation")
if plots_save:
    plt.savefig("eff_v_mom_fill" + plots_format)

In [ ]:
colors = {
    "K_DLLK>0": "xkcd:light salmon",
    "K_DLLK>5": "xkcd:red",
    "Pi_DLLK>0": "xkcd:pastel blue",
    "Pi_DLLK>5": "xkcd:blue",
}
for name, hist in hists.items():
    plt.hist(
        hist.axes[0].edges[:-1],
        bins=hist.axes[0].edges,
        weights=hist.values(),
        histtype="step",
        label=name.replace("_", " ").replace("pi", r"$\pi$"),
        color=colors[name],
        linewidth=1.5,
    )
plt.ylim(top=1.35)
plt.margins(x=-0.01)
plt.legend()
plt.xlabel("Momentum [MeV/c]")
plt.ylabel("Efficiency")
plt.figtext(0.2, 0.8, "LHCb\n $\\sqrt{s}$=13 TeV 2018 Validation")
if plots_save:
    plt.savefig("eff_v_mom_nofill" + plots_format)

## ID Efficiency vs Mis-ID Efficiency

In [ ]:
K_eff_up = [
    hists2[f"passing_K_up_{cut}"].sum().value / hists2[f"total_K_up_{cut}"].sum().value
    for cut in cuts2
]

pi_eff_up = [
    hists2[f"passing_Pi_up_{cut}"].sum().value / hists2[f"total_Pi_up_{cut}"].sum().value
    for cut in cuts2
]

K_eff_down = [
    hists2[f"passing_K_down_{cut}"].sum().value / hists2[f"total_K_down_{cut}"].sum().value
    for cut in cuts2
]

pi_eff_down = [
    hists2[f"passing_Pi_down_{cut}"].sum().value / hists2[f"total_Pi_down_{cut}"].sum().value
    for cut in cuts2
]

In [ ]:
plt.plot(K_eff_up, pi_eff_up, "s-", markersize=8, label="2018 MagUp")
plt.plot(K_eff_down, pi_eff_down, ".-", label="2018 MagDown")
plt.yscale("log")
plt.xlabel("Kaon ID Efficiency")
plt.ylabel("Pion Mis-ID Efficiency")
plt.figtext(0.2, 0.8, "LHCb\n $\\sqrt{s}$=13 TeV 2018 Validation")
plt.legend(bbox_to_anchor=(0.02, 0.8), loc="upper left")
if plots_save:
    plt.savefig("k_id_v_pi_mid_markers" + plots_format)

In [ ]:
plt.plot(K_eff_up, pi_eff_up, label="2018 MagUp")
plt.plot(K_eff_down, pi_eff_down, "--", label="2018 MagDown")
plt.yscale("log")
plt.xlabel("Kaon ID Efficiency")
plt.ylabel("Pion Mis-ID Efficiency")
plt.figtext(0.2, 0.8, "LHCb\n $\sqrt{s}$=13 TeV 2018 Validation")
plt.legend(bbox_to_anchor=(0.02, 0.8), loc="upper left")
if plots_save:
    plt.savefig("k_id_v_pi_mid_nomarkers" + plots_format)

In [ ]:
K_eff = []
pi_eff = []

mom_cuts = [3000, 10000, 20000, 50000, 100000]

for i in range(len(mom_cuts) - 1):
    K_eff.append(
        [
            hists2[f"passing_K_up_{cut}"][
                bh.loc(mom_cuts[i]) : bh.loc(mom_cuts[i + 1])
            ].sum().value
            / hists2[f"total_K_up_{cut}"][
                bh.loc(mom_cuts[i]) : bh.loc(mom_cuts[i + 1])
            ].sum().value
            for cut in cuts2
        ]
    )

    pi_eff.append(
        [
            hists2[f"passing_Pi_up_{cut}"][
                bh.loc(mom_cuts[i]) : bh.loc(mom_cuts[i + 1])
            ].sum().value
            / hists2[f"total_Pi_up_{cut}"][
                bh.loc(mom_cuts[i]) : bh.loc(mom_cuts[i + 1])
            ].sum().value
            for cut in cuts2
        ]
    )

In [ ]:
plt.plot(K_eff[0], pi_eff[0], ".-", label="p < 9.3 GeV")
plt.plot(K_eff[1], pi_eff[1], ".-", label="9.3 < p < 19 GeV")
plt.plot(K_eff[2], pi_eff[2], ".-", label="19 < p < 46 GeV")
plt.plot(K_eff[3], pi_eff[3], ".-", label="46 < p < 100 GeV")
plt.yscale("log")
plt.xlabel("Kaon ID Efficiency")
plt.ylabel("Pion Mis-ID Efficiency")
plt.figtext(0.2, 0.8, "LHCb\n $\\sqrt{s}$=13 TeV 2018 Validation")
plt.legend(bbox_to_anchor=(0.02, 0.8), loc="upper left")
if plots_save:
    plt.savefig("k_id_v_pi_mid_mom_ranges" + plots_format)